In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
japan = pd.read_csv('japan_model.csv')

In [3]:
japan.head(5)

,review,rating,date,name,tb_polarity,tb_subjectivity,vader_pos,vader_neu,vader_neg,vader_comp,...,word_yourselves,word_youth,word_yr,word_yrs,word_yum,word_yummy,word_zero,word_zone,word_zones,R_Rating
0,tokyo disneyland is truly the happiest place o...,5,2021-05-22,japan,0.485714,0.677619,0.290,0.710,0.000,0.9729,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,glad to visit not so many people on sunday th ...,5,2021-04-04,japan,0.288154,0.624656,0.208,0.792,0.000,0.9823,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,my fiance her best friend and i went here the ...,5,2020-11-28,japan,0.205093,0.351852,0.130,0.804,0.066,0.8100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,ouh mh godd we are veryy love this placee so s...,5,2020-10-17,japan,0.500000,0.566667,0.315,0.685,0.000,0.9100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,i brought my daughter to tokyo disneyland alth...,1,2020-10-16,japan,0.071000,0.429940,0.181,0.708,0.112,0.8927,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [4]:
japan.shape

(3230, 4521)

In [5]:
#!pip install spacy
#!pip install -U textblob
#!pip install vaderSentiment
import spacy
import tabulate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import path
import seaborn as sns
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix, precision_score, precision_recall_curve, recall_score, f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

### Japan data_ Modeling

In [6]:
y = japan["R_Rating"]
X = japan.drop(["R_Rating","Vader_Rating","Review_Rating", "review", "name", "date", "rating"
              ,"tb_polarity","tb_subjectivity","vader_pos","vader_neu","vader_neg","vader_comp"], axis=1)

In [7]:
X.head(5)

,n_chars,n_words,doc2vec_vector_0,doc2vec_vector_1,doc2vec_vector_2,doc2vec_vector_3,doc2vec_vector_4,word_ability,word_abit,word_able,...,word_yourself,word_yourselves,word_youth,word_yr,word_yrs,word_yum,word_yummy,word_zero,word_zone,word_zones
0,335,65,0.024619,0.347973,0.093240,-0.628225,0.273338,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,583,112,-0.288998,0.447758,-0.861468,-0.106545,-0.187401,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,500,102,0.589929,0.745137,-0.968216,-0.606862,0.278940,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,128,29,0.221496,0.880633,-0.244433,-0.818276,-0.051795,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1102,182,1.112031,0.815125,0.418955,-1.044916,0.271630,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
X.shape

(3230, 4508)

In [9]:
# split the data into train and test
from sklearn.model_selection import train_test_split

#80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### SMOTE

In [10]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

#undersampling
under = RandomUnderSampler(sampling_strategy=0.5,random_state= 42)
X_train, y_train = under.fit_resample(X_train, y_train)
steps = [('under', under)]
pipeline = Pipeline(steps=steps)

In [11]:
X_train,y_train  = pipeline.fit_resample(X_train,y_train)

In [12]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [13]:
y_train.value_counts()

1    896
0    448
Name: R_Rating, dtype: int64

### Modeling

#### Random Forest

In [14]:
#random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

In [15]:
#testing
print('Precision: %.3f' % precision_score(y_test, rf_pred))
print('Recall: %.3f' % recall_score(y_test, rf_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, rf_pred))
print('F1 Score: %.3f' % f1_score(y_test, rf_pred))

Precision: 0.854
Recall: 0.971
Accuracy: 0.842
F1 Score: 0.909


In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf_pred))

              precision    recall  f1-score   support

           0       0.69      0.28      0.40       121
           1       0.85      0.97      0.91       525

    accuracy                           0.84       646
   macro avg       0.77      0.63      0.65       646
weighted avg       0.82      0.84      0.81       646



#### XGBoost

In [17]:
import xgboost as xgb
from xgboost import XGBClassifier

xg = XGBClassifier(objective= 'binary:logistic', n_estimators = 100, random_state = 42)
xg.fit(X_train, y_train)
xg_pred = xg.predict(X_test)

In [18]:
#testing
print('Precision: %.3f' % precision_score(y_test, xg_pred))
print('Recall: %.3f' % recall_score(y_test, xg_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, xg_pred))
print('F1 Score: %.3f' % f1_score(y_test, xg_pred))

Precision: 0.911
Recall: 0.897
Accuracy: 0.845
F1 Score: 0.904


In [19]:
print(classification_report(y_test, xg_pred))

              precision    recall  f1-score   support

           0       0.58      0.62      0.60       121
           1       0.91      0.90      0.90       525

    accuracy                           0.85       646
   macro avg       0.75      0.76      0.75       646
weighted avg       0.85      0.85      0.85       646



#### XGBoost hyperparameter tuning

In [23]:
# import XGBClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

# declare parameters
parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

In [24]:
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None, nthread=4,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, seed=42,
                                     subsample=No

In [25]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=180, n_jobs=4, nthread=4, num_parallel_tree=1,
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=42, subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [29]:
xgt = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=180, n_jobs=4, nthread=4, num_parallel_tree=1,
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=42, subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [30]:
#testing
print('Precision: %.3f' % precision_score(y_test, xgt_pred))
print('Recall: %.3f' % recall_score(y_test, xgt_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, xgt_pred))
print('F1 Score: %.3f' % f1_score(y_test, xgt_pred))

Precision: 0.902
Recall: 0.910
Accuracy: 0.847
F1 Score: 0.906


In [28]:
#print(classification_report(y_test, xgt_pred))

#feature importance plot
from xgboost import plot_importance
from matplotlib import pyplot

plot_importance(xgt, max_num_features=10)
pyplot.show()

#### Logistic Regression

In [20]:
#Logistic regression classifier
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
lg.fit(X_train, y_train)

lg_pred = lg.predict(X_test)

In [21]:
print('Precision: %.3f' % precision_score(y_test, lg_pred))
print('Recall: %.3f' % recall_score(y_test, lg_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, lg_pred))
print('F1 Score: %.3f' % f1_score(y_test, lg_pred))

Precision: 0.895
Recall: 0.848
Accuracy: 0.796
F1 Score: 0.871


In [22]:
print(classification_report(y_test, lg_pred))

              precision    recall  f1-score   support

           0       0.46      0.57      0.51       121
           1       0.90      0.85      0.87       525

    accuracy                           0.80       646
   macro avg       0.68      0.71      0.69       646
weighted avg       0.81      0.80      0.80       646

